In [1]:
import pandas as pd
import sqlite3
import plotly.express as px
import numpy as np


In [2]:
# Connexion à la base de données
conn = sqlite3.connect('bdd.db')

# Récupération des données dans un DataFrame
query = """
    SELECT ObjetsTrouves.date , Temperatures.temperature, COUNT(*) as nb_objets
    FROM ObjetsTrouves
    JOIN Temperatures ON ObjetsTrouves.date = Temperatures.date
    GROUP BY ObjetsTrouves.date, Temperatures.temperature 
"""
df = pd.read_sql_query(query, conn)
df

,date,temperature,nb_objets
0,2019-01-01,8,5
1,2019-01-02,5,90
2,2019-01-03,4,107
3,2019-01-04,3,84
4,2019-01-05,4,78
...,...,...,...
1423,2022-12-27,5,84
1424,2022-12-28,9,67
1425,2022-12-29,10,65
1426,2022-12-30,10,61


In [3]:
from datetime import datetime

def get_season(date):
    # Convertir la chaîne de caractères en objet datetime
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    month = date_obj.month
    day = date_obj.day

    # Déterminer la saison en fonction du mois et du jour
    if (month == 3 and day >= 20) or (month == 4) or (month == 5) or (month == 6 and day < 21):
        return 'Printemps'
    elif (month == 6 and day >= 21) or (month == 7) or (month == 8) or (month == 9 and day < 23):
        return 'Été'
    elif (month == 9 and day >= 23) or (month == 10) or (month == 11) or (month == 12 and day < 21):
        return 'Automne'
    else:
        return 'Hiver'

df['saison'] = df['date'].apply(get_season)

In [4]:
df

,date,temperature,nb_objets,saison
0,2019-01-01,8,5,Hiver
1,2019-01-02,5,90,Hiver
2,2019-01-03,4,107,Hiver
3,2019-01-04,3,84,Hiver
4,2019-01-05,4,78,Hiver
...,...,...,...,...
1423,2022-12-27,5,84,Hiver
1424,2022-12-28,9,67,Hiver
1425,2022-12-29,10,65,Hiver
1426,2022-12-30,10,61,Hiver


In [8]:
# Scatterplot: nombre d'objets trouvés en fonction de la température
# Grouper les données par température et calculer le nombre total d'objets trouvés
grouped_data = df.groupby("temperature")["nb_objets"].sum().reset_index()

# Créer le scatterplot
scatterplot = px.scatter(grouped_data, x="temperature", y="nb_objets", title="Nombre d'objets trouvés en fonction de la température")
# Ajouter une ligne de régression pour visualiser la corrélation entre les deux variables
scatterplot.add_traces(px.scatter(grouped_data, x="temperature", y="nb_objets", trendline="ols").data[1])

# Afficher le scatterplot
scatterplot.show()

correlation = df["temperature"].corr(df["nb_objets"])
print("Coefficient de corrélation entre la température et le nombre d'objets : {:.2f}".format(correlation))

Coefficient de corrélation entre la température et le nombre d'objets : 0.10


In [6]:
grouped_data = df.groupby("saison")["nb_objets"].median().apply(np.floor).reset_index()
grouped_data
# fait un graphique pour la médiane des objets trouvés par saison
barplot = px.bar(grouped_data, x="saison", y="nb_objets", title="Nombre d'objets trouvés par saison")
barplot.show()

In [7]:
# Affichez le nombre d'objets trouvés en fonction du type de d'objet et de la saison sur un graphique. Il y a t il une correlation entre ces deux variables d'après le graphique?
query = """SELECT date , type
FROM ObjetsTrouves """

df_test = pd.read_sql_query(query, conn)
df_test['saison'] = df_test['date'].apply(get_season)
df_test = df_test.groupby(['saison', 'type'])['date'].count().reset_index()

# renomme la colonne date en nb_objets
df_test.rename(columns={'date': 'nb_objets'}, inplace=True)
# fait le graphique beau complet
barplot = px.bar(df_test, x="saison", y="nb_objets", color="type", title="Nombre d'objets trouvés par saison")
barplot.show()
